In [1]:
import csv
import pandas as pd
import math
from sklearn import cluster
import numpy as np
from scipy.spatial.distance import euclidean

In [2]:
df = pd.read_csv('checkins.dat', sep = '|', skipinitialspace=True, low_memory=False)

In [3]:
df.head()

,id,user_id,venue_id,latitude,longitude,created_at
0,984301,2041916.0,5222.0,NaN,NaN,2012-04-21 17:39:01
1,984222,15824.0,5222.0,38.895112,-77.036366,2012-04-21 17:43:47
2,984315,1764391.0,5222.0,NaN,NaN,2012-04-21 17:37:18
3,984234,44652.0,5222.0,33.800745,-84.410520,2012-04-21 17:43:43
4,984249,2146840.0,5222.0,NaN,NaN,2012-04-21 17:42:58


In [4]:
tmp = df.values
len(tmp)

1021967

In [5]:
points = []
for i in range(len(tmp)):
    if math.isnan(tmp[i][3]) or math.isnan(tmp[i][4]):
        continue
    points.append([tmp[i][3], tmp[i][4]])

print (len(points))

396634


Примечание:на 396634 строках кластеризация будет работать долго. Быть очень терпеливым не возбраняется — результат от этого только улучшится. Но для того, чтобы сдать задание, понадобится сабсет из первых 100 тысяч строк. Это компромисс между качеством и затраченным временем. Обучение алгоритма на всём датасете занимает около часа, а на 100 тыс. строк — примерно 2 минуты, однако этого достаточно для получения корректных результатов.

In [6]:
points = points[:100000]

Эта задача — хороший повод познакомиться с алгоритмом MeanShift, который мы обошли стороной в основной части лекций. Его описание при желании можно посмотреть в sklearn user guide, а чуть позже появится дополнительное видео с обзором этого и некоторых других алгоритмов кластеризации. Используйте MeanShift, указав bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

In [7]:
estimator = cluster.MeanShift(bandwidth=0.1)

In [8]:
point = estimator.fit(points)

Некоторые из получившихся кластеров содержат слишком мало точек — такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.

In [9]:
n_clusters = np.unique(estimator.labels_, return_counts=True)
clusters = [estimator.cluster_centers_[x] for x, y in zip(n_clusters[0], n_clusters[1]) if y > 15]

Как мы помним, 20 баннеров надо разместить близ офисов компании. Найдем на Google Maps по запросу Carnival Cruise Line адреса всех офисов:

In [10]:
offices = np.array([
[33.751277, -118.188740], #(Los Angeles)
[25.867736, -80.324116], #(Miami)
[51.503016, -0.075479], #(London)
[52.378894, 4.885084], #(Amsterdam)
[39.366487, 117.036146], #(Beijing)
[-33.868457, 151.205134]]) #(Sydney)

Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.

Примечание: при подсчете расстояний и в кластеризации можно пренебречь тем, что Земля круглая, так как в точках, расположенных близко друг к другу погрешность мала, а в остальных точках значение достаточно велико.

In [15]:
result = pd.DataFrame.from_records(
[[(min([euclidean(office, cluster) for cluster in clusters])),
(clusters[np.argmin(np.array([euclidean(office, cluster) for cluster in clusters]))])] for office in offices],
    columns=['distance', 'coordinate'])

Для сдачи задания выберите из получившихся 20 центров тот, который наименее удален от ближайшего к нему офиса. Ответ в этом задании — широта и долгота этого центра, записанные через пробел.

In [16]:
result.sort_values('distance', inplace = True)
print(result)

   distance                                  coordinate
5  0.007835    [-33.86063042857143, 151.20477592857145]
3  0.009353      [52.37296399032261, 4.892317222580647]
1  0.022674      [25.8456722642857, -80.31889059642857]
2  0.050058  [51.502991260887086, -0.12553728870967767]
0  0.070848     [33.8098779552631, -118.14892380690813]
4  9.267575            [31.230393000000017, 121.473704]
